In [1]:
from qiskit.circuit.library.standard_gates import UGate
import random as rnd

# Returns a circuit representation of a random unitary matrix
def random_v_qubit_unitary(v):
    rand_u = QuantumCircuit(v)
    for i in range(v):
        theta = rnd.random() * 2 * 3.14
        phi = rnd.random() * 2 * 3.14
        lamb = rnd.random() * 2 * 3.14
        name = 'RU'
        random_gate = UGate(theta, phi, lamb)
        rand_u.append(random_gate, [i])
    return rand_u

In [2]:
# Returns decimal representation of bit string (fraction)
def bin_to_dec(str):
    value = 0
    for i in range(len(str)):
        value += float(str[i]) * 2**(-1-i)
    return value

In [24]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library import QFT
from qiskit.quantum_info import Operator

def phase_estimation_circuit(t, v, U):
    """ Circuit Initialisation"""
    # Create a new circuit for phase estimation with t + v qubits.
    # We will measure the precision qubits into a classical register.
    # We therefore need t classical bits (denoted by c in the diagram).
    phase_estimation_circuit = QuantumCircuit(t+v, t)

    """ Stage 1: Preparation of State for Inverse QFT"""
    # Apply Hadamard transform to first t qubits
    phase_estimation_circuit.h(range(t))

    # Apply Controlled-U gates for each qubit in t
    for i in range(t-1, -1, -1): # t-1 to 0
        for k in range(2**i): # 0 to 2^i
            """ Create an array of indices indicating which 'wires' to connect
            the unitary to in the circuit """
            qubits = [j for j in range(t-1, t+v)]
            qubits[0] = t-i-1
            # Create a single qubit controlled-U gate from the unitary.
            cru = U.control()
            # Apply the controlled-U gate to the desired wires
            phase_estimation_circuit.append(cru, qubits)

    """ Stage 2: Applincation of Inverse QFT"""  
    # Apply inverse quantum Fourier transform
    phase_estimation_circuit.barrier(range(t+v))
    inverse_qft = QFT(t, inverse=True)
    phase_estimation_circuit.append(inverse_qft.to_gate(), range(t))

    """ Stage 3: Measurement """
    # Measure each of t qubits
    phase_estimation_circuit.barrier(range(t+v))
    phase_estimation_circuit.measure(range(t), range(t))
    
    return phase_estimation_circuit

In [1]:
import numpy as np
from qiskit.providers.aer import QasmSimulator
from qiskit import transpile

""" Tunable Parameters"""
""" Guidelines: 
- v increases matrix dimension (dimension = 2^v)
- t increases accuracy
- Keep t > v, t >= v + 2 is recommended
- Simulation runtime is exponential in t and v. Simulations of t+v > 20 take minutes to run
"""
t = 2 # number of precision qubits
v = 2 # number of eigenstate qubits
simulation_count = 100 * 2**t # Number of times to repeat the experiment

# Create a random unitary implemente as a circuit
random_unitary = random_v_qubit_unitary(v)
random_unitary = QuantumCircuit(2)
random_unitary.x(0)
random_unitary.z(1)
# Build a circuit to perform quantum phase estimation on the given unitary
QPE_circuit = phase_estimation_circuit(t, v, random_unitary)

""" Simulation """
# Create a simulator
simulator = QasmSimulator()
# Compile the circuit for the simulator
compiled_circuit = transpile(QPE_circuit, simulator)
# Simulate the circuit
simulation = simulator.run(compiled_circuit, shots=simulation_count)
# Get the simulation results
simulation_result = simulation.result()
counts = simulation_result.get_counts(compiled_circuit)

""" Processing of Simulation Results """
# Sort results from highest to lowest
results_sorted = sorted(counts, key = lambda x: abs(counts[x])**2, reverse=True)
# Get most common results
if v <= t:
    most_common_results = results_sorted[:2**v]
else:
    most_common_results = results_sorted[:2**t]    
# Get eigenvalues in a + bi form.
eigenvalue_estimates = [np.exp(2j * np.pi * bin_to_dec(k)) for i, k in enumerate(most_common_results)]

""" Obtaining Eigenvalues Using Classical Algorithm """
# Get a numpy matrix representation of the unitary gate
U = np.array(Operator(random_unitary).data)
# Calculate the eigenvalues classically
true_eigenvalues, a = np.linalg.eig(U)

ModuleNotFoundError: No module named 'qiskit.providers.aer'

In [31]:
phases = sorted([np.arccos(true_eigenvalues[i].real) / (2 * np.pi) for i in range(len(true_eigenvalues))])
phases

[0.026032763930761923,
 0.0318707050771927,
 0.03953358686782175,
 0.0547961332172749,
 0.05500049233418112,
 0.0612531418830649,
 0.0639653822081078,
 0.06829695615433724,
 0.06850131527124327,
 0.08971775829879614,
 0.09773580874655742,
 0.09794016786346336,
 0.11915661089101624,
 0.12348818483724644,
 0.12369254395415226,
 0.12620042516228966,
 0.12640478427919505,
 0.12670353714997853,
 0.132657433828078,
 0.14791998017753202,
 0.14812433929443763,
 0.15245591324066757,
 0.1551681535657105,
 0.1614208031145936,
 0.16162516223149925,
 0.18460700615793094,
 0.1981078290949927,
 0.21035938224862002,
 0.21932427212254577,
 0.2195286312394514,
 0.2238602051856816,
 0.24245405937953468,
 0.2482920005259672,
 0.25595488231659647,
 0.28539373490881675,
 0.3111461109995056,
 0.31385835132454865,
 0.31415710419533227,
 0.3143614633122383,
 0.33557790633979107,
 0.3399094802860211,
 0.34011383940292705,
 0.34262172061106433,
 0.3428260797279699,
 0.3490787292768528,
 0.3720605732032849,
 0.37

In [32]:
phases_est = sorted([np.arccos(eigenvalue_estimates[i].real) / (2 * np.pi) for i in range(len(eigenvalue_estimates))])
phases_est

[0.02734375000000002,
 0.029296875000000038,
 0.03027343750000002,
 0.06640625000000007,
 0.076171875,
 0.07910156250000003,
 0.09082031249999999,
 0.0917968750000001,
 0.09277343750000006,
 0.09326171875000011,
 0.09423828125000007,
 0.09472656250000012,
 0.09570312500000007,
 0.09765625,
 0.09863281250000008,
 0.09960937500000003,
 0.10742187500000003,
 0.12451171875,
 0.12890625000000003,
 0.12988281249999997,
 0.15478515625000006,
 0.17333984375000003,
 0.18457031250000008,
 0.18603515625000008,
 0.18798828125000003,
 0.18994140625000008,
 0.19238281250000006,
 0.2099609375,
 0.21044921875,
 0.21240234375,
 0.22021484375,
 0.23583984375,
 0.24316406250000003,
 0.26708984375,
 0.26904296875,
 0.2724609375,
 0.27294921875,
 0.27490234375,
 0.3076171875,
 0.31005859375,
 0.31396484375,
 0.3178710937500001,
 0.3217773437500001,
 0.32226562500000006,
 0.3227539062500001,
 0.32324218750000017,
 0.3256835937500001,
 0.3388671875000001,
 0.34130859375000006,
 0.34912109375000006,
 0.354492